In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from pathlib import Path

2023-04-03 20:00:23.507220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#  Import and read the charity_data.csv.
df = pd.read_csv(
    Path("Resources/charity_data.csv")
)

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(["EIN"],1)

/var/folders/qv/x2nm4qw952vblh6vpcr_nnzc0000gn/T/ipykernel_92076/264736839.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(["EIN"],1)


In [4]:
# Determine the number of unique values in each column.
df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
counts = df['NAME'].value_counts()

In [6]:
counts[counts>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [7]:
replace = list(counts[counts <= 5].index)

# Replace in dataframe
for app in replace:
    df['NAME'] = df['NAME'].replace(app,"Other")
    
# Check to make sure binning was successful
df['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [8]:
 # Look at APPLICATION_TYPE value counts for binning
app_counts = df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
 app_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in app_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
class_counts = df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
# Generate our categorical variable lists
app_categories = df.dtypes[df.dtypes == "object"].index.tolist()

In [14]:
app_with_dummies_df = pd.get_dummies(df)

In [15]:
 # Split preprocessed data into features and target arrays
X = app_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = app_with_dummies_df["IS_SUCCESSFUL"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
 # Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net
num_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=num_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               39900     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43,251
Trainable params: 43,251
Non-trainable params: 0
_________________________________________________________________


In [19]:
 # Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
 # Train model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5073 - accuracy: 0.7686
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4470 - accuracy: 0.7935
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.7948
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4312 - accuracy: 0.7985
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4280 - accuracy: 0.7985
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4256 - accuracy: 0.7987
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4238 - accuracy: 0.7995
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4223 - accuracy: 0.7988
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4212 - accuracy: 0.7997
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4201 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.4021 - accuracy: 0.8083
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4017 - accuracy: 0.8085
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4016 - accuracy: 0.8084
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4012 - accuracy: 0.8088
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4012 - accuracy: 0.8087
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4013 - accuracy: 0.8084
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4011 - accuracy: 0.8084
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4011 - accuracy: 0.8077
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4011 - accuracy: 0.8082
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4010 - accuracy: 

In [21]:
 # Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4468 - accuracy: 0.7889 - 440ms/epoch - 2ms/step
Loss: 0.4467553496360779, Accuracy: 0.7889212965965271


In [22]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")